In [20]:
import pandas as pd
import numpy as np

In [21]:
salary_data = pd.read_excel("salaire.xls", skiprows=5)
immigration_data = pd.read_excel("immigration.xls", skiprows=10)
population_data = pd.read_excel("population.xls", skiprows=5)

### First, as usual, we convert unicode to string.

In [23]:
import unicodedata

def unicode_to_string(word):
    if pd.isnull(word):
        return 
    else:
        return unicodedata.normalize('NFKD', word).encode('ascii','ignore')
    
def data_to_string(data):
    new_data = data.copy()
    
    col_title_unicode = new_data.columns.values.tolist()
    col_title = map(unicode_to_string,col_title_unicode)
    new_data.columns = col_title
    
    for col in new_data.columns:
        not_nan_index = [not ind for ind in new_data[col].isnull()]
        not_nan_value = new_data[col][not_nan_index]
        if type(not_nan_value.iloc[0]) == unicode: #check the first not-NaN value
            new_data[col] = map(unicode_to_string,new_data[col])
            
    return new_data

In [24]:
salary_data = data_to_string(salary_data)
immigration_data = data_to_string(immigration_data)
population_data = data_to_string(population_data)

### Now we will rename the column headers so that we have at least 1 common column for all the data frames. 

In [32]:
def rename_column(data): 
    new_data = data.copy()
    col_title = new_data.columns.tolist()
    for x in xrange(len(col_title)):
        if col_title[x] == 'CODGEO':
            col_title[x] = 'Code Insee' # this will be the pivot column for merging
        if col_title[x] == 'LIBGEO':
            col_title[x] = 'Libelle de la commune'
    new_data.columns = col_title
    return new_data

In [33]:
salary_data = rename_column(salary_data)
immigration_data = rename_column(immigration_data)
population_data = rename_column(population_data)

### We still have problem with departemental code of Corse because they are not in numerical form (2A... or 2B...).

In [35]:
def to_digit(string):
    new_string = list(string)[:]
    for x in xrange(len(new_string)): 
        if not new_string[x].isdigit():
            new_string[x] = '0'
    return int(''.join(new_string))

In [36]:
def replace_insee_code(data): #replace 2AXXX or 2BXXX by 20XXX 
        new_data = data.copy()
        insee_list = new_data['Code Insee'].tolist() 
        new_insee_list = [int(code) if code.isdigit() else to_digit(code) for code in insee_list]
        new_data['Code Insee'] = new_insee_list
        return new_data

In [37]:
salary_data = replace_insee_code(salary_data)
immigration_data = replace_insee_code(immigration_data)
population_data = replace_insee_code(population_data)

### Now we try to merge the 3 data frames: right_party_vote, immigration_data, salary_data

In [39]:
right_party_vote = pd.read_excel("right_party_vote.xlsx")

In [40]:
immigration_salary = pd.merge(immigration_data, salary_data, on = ['Code Insee','Libelle de la commune'], how = 'outer')

In [41]:
immigration_salary_population = pd.merge(population_data, immigration_salary, on = ['Code Insee','Libelle de la commune'], how = 'outer')

In [43]:
full_data = pd.merge(right_party_vote, immigration_salary_population, on = 'Code Insee', how = 'outer')

### Export the full data frame

In [45]:
writer = pd.ExcelWriter('full_data.xlsx')
full_data.to_excel(writer,'Sheet1')
writer.save()

In [52]:
from bokeh.io import output_file
from bokeh.models import (
  GMapPlot, GMapOptions, ColumnDataSource, Circle, DataRange1d, PanTool, WheelZoomTool, BoxSelectTool
)

map_options = GMapOptions(lat=30.29, lng=-97.73, map_type="roadmap", zoom=11)

plot = GMapPlot(
    x_range=DataRange1d(), y_range=DataRange1d(), map_options=map_options, title="Austin"
)

source = ColumnDataSource(
    data=dict(
        lat=[30.29, 30.20, 30.29],
        lon=[-97.70, -97.74, -97.78],
    )
)

circle = Circle(x="lon", y="lat", size=15, fill_color="blue", fill_alpha=0.8, line_color=None)
plot.add_glyph(source, circle)

plot.add_tools(PanTool(), WheelZoomTool(), BoxSelectTool())
output_file("gmap_plot.html")
show(plot)

ImportError: cannot import name _SessionCoordinates

In [55]:
import bokeh.io

ImportError: cannot import name _SessionCoordinates